In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# If using a remote server it is neccessary to navigate to your workspace
import os 

os.chdir('/content/drive/MyDrive/BiasMitigation/LNTL')
print('Directory changed to...',  os.getcwd())

Directory changed to... /content/drive/MyDrive/BiasMitigation/LNTL


In [3]:
# -*- coding: utf-8 -*-

# file imports
! pip install import-ipynb 
import import_ipynb
import random
from option import get_option
from trainer_merger import *
from utils.utils_LNTL import save_option
from data_loaders.data_loader_cityscapes import *
from data_loaders.data_loader_SYNTHIA import *

# torch imports
import torch
from torch.backends import cudnn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as data

# argparse jupyter workaround
from argparse import Namespace
# manually add the options becuase using juypter not command line:
# Must make it a Namespace class to be used as the argparser would be
root = '/content/drive/MyDrive/BiasMitigation/'

args = Namespace(exp_name = 'Exp1_Stage1', # Enter in format Exp1_Stage2
                 batch_size = 4,
                 momentum = 0.9,
                 optimiser = 'ADAM',  # accepts either ADAM or SGD
                 lr = 0.0001,  # try lr = 0.0001 for ADAM and lr = 0.001 SGD 
                 lr_decay_rate = 0.1,
                 lr_decay_period = 40,
                 weight_decay = 0.0001, #arbitrary value off net doubt we have time to tune. would like some paper to back up this ebing a good value
                 max_step = 100,  
                 seed = 2,
                 ##############
                 log_step = 100 ,
                 save_step = 1, # save every epoch
                 dataset = 'Cityscapes', # Accepts 'Cityscapes' or 'SYNTHIA' only
                 #  data_dir = root + 'Datasets/Cityscapes/',
                 #  meta_dir = root + 'Datasets/Cityscapes/meta',
                 save_dir = root + 'LNTL/training_logs',
                 data_split = 'train',
                 use_pretrain = False, 
                 train_baseline = False,
                 train_greyscale = False,
                 train_bias_head_only = False,
                 #############
                 checkpoint = None, # root + 'LNTL/training_logs/Deeplab/Cityscapes/LNTL/Exp5_Stage1/checkpoints/checkpoint_bias_head_epoch_0039.pth', 
                 load_bias_head_only = False,
                 load_seg_head_only = False,
                 #############
                 val_only_greyscale = False,
                 val_only_jitter = False,
                 val_only_invert = False,
                 #############
                 random_seed = None,
                 num_workers = 2,
                 cudnn_benchmark = True,
                 #############
                 cuda = True ,
                 debug = False, 
                 is_train = True,  
                 #############
                 network_type = 'SegNet',  # Enter only Deeplab or SegNet
                 notes = 'SegNet within the LNTL scheme. Input and validation are colour images from the dataloaders'
                 )


'''
parser.add_argument('-e', '--exp_name',   required=True,              help='experiment name')

parser.add_argument('--n_class',          default=10,     type=int,   help='number of classes')
parser.add_argument('--input_size',       default=28,     type=int,   help='input size')
parser.add_argument('--batch_size',       default=128,    type=int,   help='mini-batch size')
parser.add_argument('--momentum',         default=0.9,    type=float, help='sgd momentum')
parser.add_argument('--lr',               default=0.01,   type=float, help='initial learning rate')
parser.add_argument('--lr_decay_rate',    default=0.1,    type=float, help='lr decay rate')
parser.add_argument('--lr_decay_period',  default=40,     type=int,   help='lr decay period')
parser.add_argument('--weight_decay',     default=0.0005, type=float, help='sgd optimizer weight decay')
parser.add_argument('--max_step',         default=100,    type=int,   help='maximum step for training')
parser.add_argument('--depth',            default=20,     type=int,   help='depth of network')
parser.add_argument('--color_var',        default=0.03,   type=float, help='variance for color distribution')
parser.add_argument('--seed',             default=2,      type=int,   help='seed index')


parser.add_argument('--checkpoint',       default=None,               help='checkpoint to resume')
parser.add_argument('--log_step',         default=50,     type=int,   help='step for logging in iteration')
parser.add_argument('--save_step',        default=10,     type=int,   help='step for saving in epoch')
parser.add_argument('--data_dir',         default='./',               help='data directory')
parser.add_argument('--save_dir',         default='./',               help='save directory for checkpoint')
parser.add_argument('--data_split',       default='train',            help='data split to use')
parser.add_argument('--use_pretrain',     action='store_true',        help='whether it use pre-trained parameters if exists')
parser.add_argument('--train_baseline',   action='store_true',        help='whether it train baseline or unlearning')
parser.add_argument('--train_greyscale',  action='store_true',        help='whether to convert the images to greyscale to get a measure of no colour information')

parser.add_argument('--random_seed',                      type=int,   help='random seed')
parser.add_argument('--num_workers',      default=4,      type=int,   help='number of workers in data loader')
parser.add_argument('--cudnn_benchmark',  default=True,   type=bool,  help='cuDNN benchmark')


parser.add_argument('--cuda',             action='store_true',        help='enables cuda')
parser.add_argument('-d', '--debug',      action='store_true',        help='debug mode')
parser.add_argument('--is_train',         action='store_true',        help='whether it is training')
'''

def log_file_setup(option):
    # setting up training log files:
    if option.train_baseline:
        if option.train_greyscale:
            kind = 'baseline_greyscale'
            path = option.save_dir + '/' + option.network_type + '/' + option.dataset + '/baseline' + '/greyscale' + '/' + option.exp_name 
            if not os.path.exists(path):
                os.makedirs(path)
        else:
            kind = 'baseline_colour'
            # path = os.path.join(option.save_dir, '/' , option.network_type, '/', option.dataset, '/baseline', '/colour', '/', option.exp_name )
            path = option.save_dir + '/' + option.network_type + '/' + option.dataset + '/baseline' + '/colour' + '/' + option.exp_name
            if not os.path.exists(path):
                os.makedirs(path)
    else:
        kind = 'LNTL'
        #path = os.path.join(option.save_dir, '/' , option.network_type, '/', option.dataset, '/LNTL', '/', option.exp_name )
        path = option.save_dir + '/' + option.network_type + '/' + option.dataset + '/LNTL' + '/' + option.exp_name
        if not os.path.exists(path):
            os.makedirs(path)
    
    return path


def backend_setting(option):

    if option.random_seed is None:
        option.random_seed = random.randint(1,10000)
    torch.manual_seed(option.random_seed)

    if torch.cuda.is_available() and not option.cuda:
        print('WARNING: GPU is available, but we are not using it')

    if not torch.cuda.is_available() and option.cuda:
        option.cuda = False
        print('Warning: Youve asked for CUDA but no GPU is available...setting CUDA to False')

    if option.cuda:
        os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
        #os.environ['CUDA_VISIBLE_DEVICES'] = ','.join([str(i) for i in option.gpu_ids])
        torch.cuda.manual_seed_all(option.random_seed)
        cudnn.benchmark = option.cudnn_benchmark
    
    if option.train_baseline:
        option.is_train = True


def main():
    #option = get_option()
    option = args
    #debug:
    #print(option)
    
    path = log_file_setup(option)
    backend_setting(option)
    trainer = Trainer(option, path)

    if option.dataset == 'Cityscapes':
        train_dataset = DatasetTrain(option)
        val_dataset = DatasetVal(option)
    elif option.dataset == 'SYNTHIA':
        train_dataset = DatasetTrain_SYNTHIA(option)
        val_dataset = DatasetVal_SYNTHIA(option)


    train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                                batch_size=option.batch_size,
                                                shuffle=True,
                                                num_workers=option.num_workers)

    val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                                                batch_size=option.batch_size,
                                                shuffle=False,
                                                num_workers=option.num_workers)

    if option.is_train:   
        save_option(option, path)
        trainer.train(train_loader = train_loader, val_loader = val_loader)
    else:
        trainer._validate(trainval_loader) # I dont think we need this

        pass

if __name__ == '__main__': main()

importing Jupyter notebook from option.ipynb
importing Jupyter notebook from trainer_merger.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Changed the local path to.... /content/drive/MyDrive/BiasMitigation/LNTL
importing Jupyter notebook from /content/drive/MyDrive/BiasMitigation/LNTL/models/Deeplab/deeplabv3.ipynb
importing Jupyter notebook from /content/drive/MyDrive/BiasMitigation/LNTL/models/Deeplab/resnet.ipynb
importing Jupyter notebook from /content/drive/MyDrive/BiasMitigation/LNTL/models/Deeplab/aspp.ipynb
importing Jupyter notebook from /content/drive/MyDrive/BiasMitigation/LNTL/models/SegNet/segnet.ipynb
importing Jupyter notebook from /content/drive/MyDrive/BiasMitigation/LNTL/models/biashead.ipynb
importing Jupyter notebook from /content/drive/MyDrive/BiasMitigation/LNTL/utils/utils_LNTL.ipynb
importing Jupyter notebook from /content/drive/MyDrive/BiasMitigation/LNTL/utils/utils_Deepla

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:693: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch._C._nn.max_pool2d_with_indices(input, kernel_size, stride, padding, dilation, ceil_mode)
[Exp1_Stage1] INFO: [TRAIN] cls loss : 3.246689 (epoch 0.00)
[Exp1_Stage1] INFO: [TRAIN] cls loss : 3.054026 (epoch 0.13)
[Exp1_Stage1] INFO: [TRAIN] cls loss : 2.443471 (epoch 0.26)
[Exp1_Stage1] INFO: [TRAIN] cls loss : 2.214620 (epoch 0.40)
[Exp1_Stage1] INFO: [TRAIN] cls loss : 1.851205 (epoch 0.53)
[Exp1_Stage1] INFO: [TRAIN] cls loss : 1.824208 (epoch 0.67)
[Exp1_Stage1] INFO: [TRAIN] cls loss : 2.017365 (epoch 0.80)
[Exp1_Stage1] INFO: [TRAIN] cls loss : 2.099169 (epoch 0.94)


Training epoch loss:.... 2.2838106


[Exp1_Stage1] INFO: [VAL] cls loss : 2.075267 (epoch 0.00)
[Exp1_Stage1] INFO: [VAL] cls loss : 1.854570 (epoch 0.80)


Validation epoch loss:.... 2.1713488
Checkpoint Saved. Epoch : 0


...

Validation epoch loss:.... 1.0284187
Checkpoint Saved. Epoch : 98


[Exp1_Stage1] INFO: [TRAIN] cls loss : 0.918785 (epoch 99.00)
[Exp1_Stage1] INFO: [TRAIN] cls loss : 1.092088 (epoch 99.13)
[Exp1_Stage1] INFO: [TRAIN] cls loss : 1.398526 (epoch 99.26)
[Exp1_Stage1] INFO: [TRAIN] cls loss : 0.873689 (epoch 99.40)
[Exp1_Stage1] INFO: [TRAIN] cls loss : 0.956669 (epoch 99.53)
[Exp1_Stage1] INFO: [TRAIN] cls loss : 1.166643 (epoch 99.67)
[Exp1_Stage1] INFO: [TRAIN] cls loss : 0.498724 (epoch 99.80)
[Exp1_Stage1] INFO: [TRAIN] cls loss : 1.709062 (epoch 99.94)


Training epoch loss:.... 0.891109


[Exp1_Stage1] INFO: [VAL] cls loss : 0.929890 (epoch 99.00)
[Exp1_Stage1] INFO: [VAL] cls loss : 0.685120 (epoch 99.80)


Validation epoch loss:.... 1.0178503
Checkpoint Saved. Epoch : 99
